In [ ]:
import os
import pandas as pd
import re
from tqdm import tqdm
from utils import load_jsonl_iteratively

iter_losses = []
task_names = {
    'ja_knowledge_memorization': 'AdaXEval-Ja (mem)',
    # 'ja_knowledge_generalization': 'AdaXEval-Ja (gen)',
    # 'en_knowledge_memorization': 'AdaXEval-En (mem)',
    # 'en_knowledge_generalization': 'AdaXEval-En (gen)'
}
    
# ROOT = "/model/projects/cpt-scidoc/roman-pretrain/exp1-en_jstage"
ROOT = "/data/xzhao/experiments/roman-pretrain/exps/exp1-multi-ja/hf_models"

In [4]:

examples = {}
for fn in tqdm(os.listdir(ROOT)):
    if not fn.startswith("iter_"):
        continue

    iter_info = {task: [] for task in task_names}
    num_iter = int(re.match("iter_(\d+)", fn).group(1))
    fn = os.path.join(ROOT, fn, "result.json")
    
    if not os.path.exists(fn):
        continue
    
    task2sample_ids = {task: set() for task in task_names}
    for item in load_jsonl_iteratively(fn):
        sample = item['sample']
        if 'dataset' not in sample:
            continue
        task = sample['dataset']
        if task not in task_names:
            continue
        
        sample_id = sample['sample_id']
        if sample_id in task2sample_ids[task]:
            continue
        
        task2sample_ids[task].add(sample_id)

        answer_idx = int(sample['answer_idx'])
        valid_loss = item['losses'][answer_idx]
        valid_normloss = item['norm_losses'][answer_idx]
        invalid_loss = sum([item['losses'][i] for i in range(4) if i != answer_idx])/3
        invalid_normloss = sum([item['norm_losses'][i] for i in range(4) if i != answer_idx])/3

        iter_info[task].append((sample_id, valid_loss, valid_normloss, invalid_loss, invalid_normloss))

    # for task in task2sample_ids:
    #     print("Iter {}: Task {} has {} samples".format(num_iter, task_names[task], len(task2sample_ids[task])))
    
    for task, losses in iter_info.items():
        sample_id, valid_loss, valid_normloss, invalid_loss, invalid_normloss = list(zip(*losses))

        iter_losses.append({
            'sample_id': sample_id,
            'num_iter': num_iter,
            'task': task_names[task],
            'valid_loss': sum(valid_loss)/len(valid_loss),
            'valid_normloss': sum(valid_normloss)/len(valid_normloss),
            'invalid_loss': sum(invalid_loss)/len(invalid_loss),
            'invalid_normloss': sum(invalid_normloss)/len(invalid_normloss),
            'valid_loss_ratio': sum(valid_loss) / (sum(valid_loss) + 3 * sum(invalid_loss)),
            'valid_normloss_ratio': sum(valid_normloss) / (sum(valid_normloss) + 3 * sum(invalid_normloss)),
            'num_sample': len(losses),
        })
df = pd.DataFrame(iter_losses)

100%|██████████| 20/20 [00:00<00:00, 4149.49it/s]


In [5]:
df

""


In [26]:
lang = "ja"
import random
sample_ids = list(iter2task2sample_ids["3000"][f'{lang}_knowledge_memorization'].keys())
sample_id = random.sample(sample_ids, 1)[0]
task2samples[f'{lang}_knowledge_memorization'][sample_id]


{'sample_id': 'jamt@@65/1/65_15-33_sentid:1',
 'question': 'HBs抗原は [BLANK] である。',
 'options': ['HBVの表面糖鎖', 'HBVのエンベロープ蛋白', 'HBVのコア蛋白', 'HBVのDNAポリメラーゼ'],
 'answer_idx': 1,
 'n_options': 4,
 'metadata': {'docid': 'jamt@@65/1/65_15-33',
  'sentid': '1',
  'correct_answer': 'HBVのエンベロープ蛋白',
  'triple': {'subject': 'HBs抗原',
   'relation': 'は...である',
   'object': 'HBVのエンベロープ蛋白'},
  'sentence': 'HBs抗原は,HBVのエンベロープ蛋白であるが,従来の診断マーカーとしての定性検査に加え,近年HBs抗原量がB型肝炎治療の効果判定や予後予測における有用なマーカーとなることが報告され,その定量測定が臨床上重要となっている。'},
 'dataset': 'ja_knowledge_memorization'}

In [27]:
iterations = sorted(iter2task2sample_ids.keys(), key=lambda x: int(x))
for iteration in iterations:
    item = iter2task2sample_ids[iteration][f'{lang}_knowledge_memorization'][sample_id]
    print(iteration, item['losses'], item['valid_loss'])

task2samples[f'{lang}_knowledge_memorization'][sample_id]


0 [42.75, 38.25, 38.25, 44.75] 38.25
10 [43.5, 39.25, 39.0, 45.75] 39.25
100 [43.75, 39.0, 38.5, 44.0] 39.0
500 [47.25, 43.25, 44.25, 47.0] 43.25
1000 [51.25, 41.0, 42.75, 47.5] 41.0
2000 [50.5, 43.5, 44.75, 48.0] 43.5
3000 [53.25, 46.25, 44.25, 50.25] 46.25
4000 [53.25, 46.25, 47.0, 51.25] 46.25
5000 [57.0, 44.75, 46.5, 51.5] 44.75
6000 [57.25, 47.75, 47.25, 50.25] 47.75
7000 [63.25, 49.25, 50.25, 57.25] 49.25
8000 [66.5, 55.25, 50.75, 60.5] 55.25
9000 [62.75, 51.5, 50.25, 57.75] 51.5
10000 [62.75, 50.75, 50.25, 57.5] 50.75
11000 [69.0, 55.5, 54.5, 58.25] 55.5
12000 [72.0, 58.5, 57.0, 61.0] 58.5
13000 [71.5, 59.75, 56.25, 62.5] 59.75
14000 [71.5, 58.75, 57.5, 62.0] 58.75
15000 [74.0, 58.25, 57.0, 63.75] 58.25
15364 [74.0, 58.75, 56.75, 63.5] 58.75


{'sample_id': 'jamt@@65/1/65_15-33_sentid:1',
 'question': 'HBs抗原は [BLANK] である。',
 'options': ['HBVの表面糖鎖', 'HBVのエンベロープ蛋白', 'HBVのコア蛋白', 'HBVのDNAポリメラーゼ'],
 'answer_idx': 1,
 'n_options': 4,
 'metadata': {'docid': 'jamt@@65/1/65_15-33',
  'sentid': '1',
  'correct_answer': 'HBVのエンベロープ蛋白',
  'triple': {'subject': 'HBs抗原',
   'relation': 'は...である',
   'object': 'HBVのエンベロープ蛋白'},
  'sentence': 'HBs抗原は,HBVのエンベロープ蛋白であるが,従来の診断マーカーとしての定性検査に加え,近年HBs抗原量がB型肝炎治療の効果判定や予後予測における有用なマーカーとなることが報告され,その定量測定が臨床上重要となっている。'},
 'dataset': 'ja_knowledge_memorization'}

{'sample_id': 'haigan1960@@31/1/31_1_1_sentid:5',
 'question': '近年、扁平上皮癌と腺癌の発生頻度の関係はどのように変化していると考えられていますか？',
 'options': ['腺癌の方が多くなっている',
  '両者とも発生率が減少している',
  '扁平上皮癌の方がさらに増加している',
  '扁平上皮癌と腺癌の発生率がほぼ同じになっている'],
 'answer_idx': 0,
 'n_options': 4,
 'metadata': {'docid': 'haigan1960@@31/1/31_1_1',
  'sentid': '5',
  'correct_answer': '腺癌の方が多くなっている',
  'triple': {'subject': '扁平上皮癌',
   'relation': 'が腺癌より多い状況が変化し',
   'object': '腺癌の方が多くなっている'},
  'sentence': '従来, 扁平上皮癌が腺癌より多いと言われていたが, 近年では, 腺癌の方が多くなっている可能性が示唆された.'},
 'dataset': 'ja_knowledge_generalization'}